In [1]:
import pandas as pd
import altair as alt
import duckdb
import os

DATA_DIR = os.getenv("DATA_DIR", "../data")

from IPython.display import Markdown

_ = alt.renderers.set_embed_options(theme="ggplot2", actions=False)

%load_ext sql
con = duckdb.connect(f"{DATA_DIR}/dbt.duckdb", read_only=True)
%sql con --alias duckdb


In [14]:
storage_provider_power_df = con.table("filecoin_storage_providers_power").df()
storage_provider_power_df["date"] = pd.to_datetime(storage_provider_power_df["date"])
storage_provider_power_df.sample(5)


,date,provider_id,raw_power_bytes,raw_power_pibs,quality_adjusted_power_bytes,quality_adjusted_power_pibs,verified_data_power_bytes,verified_data_power_pibs,raw_power_pibs_bucket,verified_data_power_pibs_bucket,country
2114644,2022-12-26,f080103,285014029762560,0.253143,316728575623168,0.281311,3.523838e+12,0.00313,>0.1<1,<0.1,KR
2118133,2022-12-25,f014302,0,0.000000,0,0.000000,0.000000e+00,0.00000,=0,=0,HK
5753585,2021-10-29,f0214334,1099958304374780,0.976959,1099958304374780,0.976959,0.000000e+00,0.00000,>0.1<1,=0,KR
5335806,2021-12-24,f029632,0,0.000000,0,0.000000,0.000000e+00,0.00000,=0,=0,DE
4784100,2022-03-03,f0125962,0,0.000000,0,0.000000,0.000000e+00,0.00000,=0,=0,KR


In [15]:
# | output: asis

print(
    f""" 
Updated on {storage_provider_power_df['date'].max().strftime('%Y-%m-%d')}.
"""
)


 
Updated on 2023-07-25.



In [17]:
latest_state = storage_provider_power_df[
    storage_provider_power_df["date"] == storage_provider_power_df["date"].max()
]


::: {layout="[ [1,1,1]]"}

In [18]:
# | output: asis

unique_miners_with_active_power = latest_state[
    latest_state["raw_power_pibs"] > 0
].nunique()["provider_id"]

print(
    f"""
### Active Storage Providers
{unique_miners_with_active_power}
"""
)



### Active Storage Providers
3479



In [19]:
# | output: asis

print(
    f"""
### Total Raw Power
{latest_state['raw_power_pibs'].sum():.2f} PiBs
"""
)



### Total Raw Power
20956.99 PiBs



In [21]:
# | output: asis

print(
    f"""
### Total Quality Adjusted PiB Power
{latest_state['quality_adjusted_power_pibs'].sum():.2f} PiBs
"""
)



### Total Quality Adjusted PiB Power
33702.90 PiBs



:::

::: {layout="[ [1,1], [1] ]"}


In [29]:
# | output: true

display(Markdown(f"### Storage Provider Raw PiB Power"))

c = (
    alt.Chart(
        storage_provider_power_df.groupby("date").sum()["raw_power_pibs"].reset_index()
    )
    .mark_area()
    .encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("raw_power_pibs:Q", title="Raw PiB Power"),
        tooltip=["date", "raw_power_pibs"],
        color=alt.Color("raw_power_pibs_bucket:N", title="Raw PiB Power Bucket"),
    )
    .properties(width="container")
)

c.display()


### Storage Provider Raw PiB Power

alt.Chart(...)

In [9]:
# | output: true

display(Markdown(f"### Storage Provider Quality Adjusted PiB Power"))

c = (
    alt.Chart(
        storage_provider_power_df.groupby("stat_date")
        .sum()["quality_adjusted_pib_power"]
        .reset_index()
    )
    .mark_area()
    .encode(
        x=alt.X("stat_date:T", title="Date"),
        y=alt.Y("quality_adjusted_pib_power:Q", title="Quality Adjusted PiB Power"),
        tooltip=["stat_date", "quality_adjusted_pib_power"],
        color=alt.value("#55595c"),
    )
    .properties(width="container")
)

c.display()


### Storage Provider Quality Adjusted PiB Power

alt.Chart(...)

In [10]:
# | output: true

display(Markdown(f"### Storage Providers With Power over Time"))

c = (
    alt.Chart(
        storage_provider_power_df[storage_provider_power_df["raw_byte_power"] > 0]
        .groupby("stat_date")
        .agg({"miner_id": "nunique"})
        .reset_index()
        .rename(columns={"stat_date": "date", "miner_id": "unique_miners"})
    )
    .mark_area()
    .encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("unique_miners:Q", title="Unique Miners"),
        tooltip=["date", "unique_miners"],
        color=alt.value("#55595c"),
    )
    .properties(width="container")
)

c.display()


### Storage Providers With Power over Time

alt.Chart(...)

:::

::: {layout="[ [1,1], [1] ]"}

In [12]:
# | output: true

display(
    Markdown(f"### Storage Provider Raw PiB Power Growth Rate (7 Days Rolling Mean)")
)

daily_pib_power = (
    storage_provider_power_df.groupby("stat_date").sum()["raw_pib_power"].reset_index()
)

daily_pib_power = daily_pib_power.assign(
    growth_rate=lambda df: df["raw_pib_power"].pct_change(periods=7).rolling(7).mean()
    * 100
)

c = (
    alt.Chart(daily_pib_power)
    .mark_line()
    .encode(
        x=alt.X("stat_date:T", title="Date"),
        y=alt.Y("growth_rate:Q", title="Raw PiB Power Growth Rate"),
        tooltip=["stat_date", "growth_rate"],
        color=alt.value("#55595c"),
    )
    .properties(width="container")
    .interactive(bind_y=False)
)

line = (
    alt.Chart(pd.DataFrame({"y": [0]}))
    .mark_rule()
    .encode(y="y", color=alt.value("#999"))
)

(c + line).display()


### Storage Provider Raw PiB Power Growth Rate (7 Days Rolling Mean)

alt.LayerChart(...)

In [13]:
# | output: true

display(
    Markdown(
        f"### Storage Providers With Power over Time Growth Rate (7 Days Rolling Mean)"
    )
)

daily_miners_with_power = (
    storage_provider_power_df[storage_provider_power_df["raw_byte_power"] > 0]
    .groupby("stat_date")
    .agg({"miner_id": "nunique"})
    .reset_index()
    .rename(columns={"stat_date": "date", "miner_id": "unique_miners"})
)

daily_miners_with_power = daily_miners_with_power.assign(
    growth_rate=lambda df: df["unique_miners"].pct_change(periods=7).rolling(7).mean()
    * 100
)

c = (
    alt.Chart(daily_miners_with_power)
    .mark_bar()
    .encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("growth_rate:Q", title="Unique Miners Growth Rate"),
        tooltip=["date", "growth_rate"],
        # Color red if negative, green if positive
        color=alt.condition(
            alt.datum.growth_rate > 0,
            alt.value("#28a745"),  # The positive color
            alt.value("#dc3545"),  # The negative color
        ),
    )
    .properties(width="container")
    # Display line at x axis 0
).interactive(bind_y=False)

line = (
    alt.Chart(pd.DataFrame({"y": [0]}))
    .mark_rule()
    .encode(y="y", color=alt.value("#999"))
)

(c + line).display()


### Storage Providers With Power over Time Growth Rate (7 Days Rolling Mean)

alt.LayerChart(...)

:::